In [ ]:
import notebook_header
from header import Product, ProductFields, Supplier, start_supplier
s = start_supplier('kualastyle')
""" s - на протяжении всего кода означает класс `Supplier` """


from dict_scenarios import scenario
s.current_scenario = scenario['Sofas and Sectionals']

# -------------- сокращения -------
l = s.reread_locators('product')
d = s.driver
_ = d.execute_locator
# --------------------------------
from list_products_in_category import product_urls

In [ ]:
# -*- coding: utf-8 -*-


""" Я добавляю в базу данных престашоп товар путем нескольких последовательных действий
1. Заполняю поля, необходимые для создания нового товара
2. Получаю `id_product` созданного товара
3. Используя полученный `id_product` загружаю дефолтную картинку
4. итд.
"""

from typing import Union
# ----------------------------
from global_settings import gs
from src.logger.logger import logger
from src.tools import StringFormatter, StringNormalizer
from src.product import Product, ProductFields
from src.suppliers import Supplier
# ----------------------------




def grab_product_page(s: Supplier, id_product: int = 0 , presta_api_version: Union[str('V1'),str('V2'),str('V3')] = 'V3') -> ProductFields:
    """ ПОКА ТОЛЬКО ДЛЯ НОВЫХ!!!!! СТРАНИЦ
        Собираю информацию со страницы товара. 
        Важно помнить, что драйвер уже должен быть на
        этой странице
        ---------------
        Attrs:
            s (Supplier)
            id_product `int`  :  if 0 -> new product
        @returns
            f (ProductFields) с заполненными полями, else False
    """


    l = s.reread_locators('product')
    d = s.driver
   
    _ = d.execute_locator

    print('Starting Product grabber')

    def add_new_product_stage_1(s: Supplier) -> ProductFields:
        """ Первая стадия добавления нового товара. Заполняю все необходимые поля
            Далее я отправлю их в PrestaShop и получу обратно ID вновь созданного товара
            На второй стадии, зная ID, я отправлю главную картинку и прочее
        """
        f: ProductFields = ProductFields()
       
        """ ID,ASIN,SKU,SUPPLIER SKU """
    

        def _set_defaults() -> bool:
            """ Set defaults for product of supplier """
            f.active = 1
            f.on_sale = 1
            f.min_qty = 1
            f.low_stock_level = 0
            f.low_stock_threshold = ''
            f.show_price = 1
            f.show_condition = 1
            f.aviable_online_only = 0
            f.advanced_stock_management = 0
            f.state = 1

        def set_price(s, format: str = 'str') -> Union[str,float]:
            """ Привожу денюшку к 
            [ ] float 
            [v] str
            """
            
            raw_price = _(l['price'])[0]
            #raw_price = str(raw_price).split('\n')[0]
            return StringNormalizer.normalize_price(raw_price)
    

        """ Я добавляю в базу данных престашоп товар путем нескольких последовательных действий
        1. Добавляю поля, необходимые для создания нового товара
        2. Получаю `id_product` созданного товара
        3. Используя полученный `id_product` загружаю дефолтную картинку
        4. итд.
        """

        _set_defaults()
        l = s.reread_locators('product')
        reference = _(l['reference'])[0]
        reference = StringFormatter.clear_numbers(reference)
        f.reference = f'{s.supplier_id}-{reference}'
        f.supplier_reference = reference
        f.price = set_price(s, format = 'str')
        f.name = StringNormalizer.normilize_string(_(l['name']))
        
        f.description_short = StringNormalizer.normilize_string(_(l['description_short'])[0])
        f.id_supplier = s.supplier_id
        
        _category_default = list(s.current_scenario['presta_categories']['default_category'].keys())[0]
        f.id_category_default = _category_default
        f.category_ids = Product.get_parent_categories(_category_default)

        #affiliate = _(l['affiliate short link'])
        #affiliate = affiliate[1][0]
        #f.affiliate_short_link = affiliate

        f.link_rewrite = reference



        #f.link_rewrite = "test-link"
        return f

    def upload_image(id_product, image_url) -> ProductFields:
        """ После того, как я занес новый товар в бд - я получу его id
        Далее я гружу фото и получаю ее id
        Далее я догружаю осталные па
        ----------------------
        Attrs: 
            s (Supplier):
            f (ProductFields): Заполненные на первом этапе поля. Я беру из них только то, что мне надо для апдейта
        @returns
            ProductFields: поля для апдейта
        """

        img = Product.upload_product_default_image(id_product, image_url)
        return img
        
    



    if id_product == 0:
        print("---------------------NEW PRODUCT-----------------------")
        f = add_new_product_stage_1(s)
        product_dict = f.product_dict
        pprint(product_dict)
        product = Product.add_2_PrestaShop(product_dict, presta_api_version)
        id_product = product['id']
        print("---------------------NEW PRODUCT ID-----------------------")
        print(id_product)
        image_url = _(l['additional_images_urls'])[0]
        img = upload_image(id_product, image_url)
        return product, img

    
    ...


In [ ]:
reference = _(l['reference'])
pprint(reference)
x = StringNormalizer.normilize_string( _(l['name']))
pprint(x)

In [ ]:
image_url = _(l['additional_images_urls'])